In [1]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GlobalAveragePooling1D, Lambda, Concatenate
from tensorflow.keras.layers import concatenate
from keras.models import Model
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

# from BertEmbeddings import BertEmbeddings
from transformers import RobertaTokenizer, TFRobertaModel
from transformers import DebertaV2Tokenizer, TFDebertaV2Model

In [90]:
print(Train_Data_File)

train_with_features (cleaned).csv


In [99]:
# Define constants and parameters

# Supported data_clean_type (DO NOT forget to put on "()", also, if data_clean_type is NOT empty string, please put a " " before (cleaned)):
# empty string, no character
# (cleaned)
# (cleaned)(hyper_cleaned)
# (cleaned)(hyper_cleaned)(punctuation_removed)
# (cleaned)(hyper_cleaned)(punctuation_removed)(stopwords_removed)
# (cleaned)(hyper_cleaned)(punctuation_removed)(stopwords_removed)(words_shortened)
# (cleaned)(hyper_cleaned)(punctuation_removed)(stopwords_removed)(alternative_stopwords_used)
# (cleaned)(hyper_cleaned)(punctuation_removed)(stopwords_removed)(alternative_stopwords_used)(words_shortened)
data_clean_type = " (cleaned)(hyper_cleaned)(punctuation_removed)(stopwords_removed)(alternative_stopwords_used)(words_shortened)"

Train_Data_File = 'train_with_features' + data_clean_type + '.csv'
Test_Data_File = 'test_with_features' + data_clean_type + '.csv'
Max_Sequence_Length = 60
Max_Num_Words = 200000
Validation_Split_Ratio = 0.2

Num_Lstm = np.random.randint(175, 275)
Num_Dense = np.random.randint(100, 150)
Rate_Drop_Lstm = 0.15 + np.random.rand() * 0.25
Rate_Drop_Dense = 0.15 + np.random.rand() * 0.25

act_f = 'relu'
re_weight = False
use_more_features = False
fine_tuned_model = False

# embedding_method:
# 1. Bert (will use Bert large)
# 2. DeBerta (will use DeBerta-v2-xlarge)
# 3. GloVe
# 4. RoBerta (will use RoBerta-large)
# 5. Word2Vec
embedding_model = "GloVe"

# Support neural network:
# LSTM
# CNN
neural_network = "LSTM"

In [252]:
# Process text in dataset
print('Processing text dataset')

# load data and process with text_to_wordlist
df_train = pd.read_csv(Train_Data_File, encoding='utf-8')
#df_train = df_train.dropna()
#df_train = df_train.fillna('empty')

df_train = df_train[['question1', 'question2', 'is_duplicate', 'q1_q2_intersect', 'q1_freq', 'q2_freq']]
df_train = df_train.dropna()

# train_texts_1 = df_train['question1'].tolist()
# train_texts_2 = df_train['question2'].tolist()
# train_labels = df_train['is_duplicate'].tolist()

df_test = pd.read_csv(Test_Data_File, encoding='utf-8')
df_test = df_test.dropna()
#df_test = df_test.fillna('empty')

test_texts_1 = df_test['question1'].tolist()
test_texts_2 = df_test['question2'].tolist()
test_ids = df_test['test_id'].tolist()

Processing text dataset


C:\Users\LTX_\AppData\Local\Temp\ipykernel_30008\994337999.py:5: DtypeWarning: Columns (5,6,7,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(Train_Data_File, encoding='utf-8')


0


In [269]:
# ignore all non 0 or 1 val in train_val
temp_train = []
isDup = df_train['is_duplicate'].tolist()

for s in range (len(df_train)):
    if (isDup[s] == '0' or isDup[s] == '1'):
        temp_train.append(s)
        


In [271]:
df_train = df_train[['question1', 'question2', 'is_duplicate', 'q1_q2_intersect', 'q1_freq', 'q2_freq']].iloc[temp_train]

In [272]:
df_train = df_train.dropna()

train_texts_1 = df_train['question1'].tolist()
train_texts_2 = df_train['question2'].tolist()
train_labels = df_train['is_duplicate'].tolist()

In [275]:
train_labels = [int(numeric_string) for numeric_string in train_labels]

In [278]:

print(len(test_texts_1))

4288


In [279]:
# Tokenize words in all sentences
tokenizer = Tokenizer(num_words=Max_Num_Words)
tokenizer.fit_on_texts(train_texts_1 + train_texts_2 + test_texts_1 + test_texts_2)

train_sequences_1 = tokenizer.texts_to_sequences(train_texts_1)
train_sequences_2 = tokenizer.texts_to_sequences(train_texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print('{} unique tokens are found'.format(len(word_index)))

# pad all train with Max_Sequence_Length
train_data_1 = pad_sequences(train_sequences_1, maxlen=Max_Sequence_Length)
train_data_2 = pad_sequences(train_sequences_2, maxlen=Max_Sequence_Length)
train_labels = np.array(train_labels)
print('Shape of train data tensor:', train_data_1.shape)
print('Shape of train labels tensor:', train_labels.shape)

# pad all test with Max_Sequence_Length
test_data_1 = pad_sequences(test_sequences_1, maxlen=Max_Sequence_Length)
test_data_2 = pad_sequences(test_sequences_2, maxlen=Max_Sequence_Length)
test_ids = np.array(test_ids)
print('Shape of test data tensor:', test_data_2.shape)
print('Shape of test ids tensor:', test_ids.shape)

58424 unique tokens are found
Shape of train data tensor: (311149, 60)
Shape of train labels tensor: (311149,)
Shape of test data tensor: (4288, 60)
Shape of test ids tensor: (4288,)


In [284]:
print(train_texts_2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [208]:
# Word embedding
if embedding_model == "GloVe":
    Embedding_Dim = 300 # Dimension of GloVe-embedding
    Embedding_File = 'glove.840B.300d.txt' # You may need to add the complete directory if the text file is NOT in the same folder as this ipynb file
    # Create word embedding dictionary from 'glove.840B.300d.txt'
    print('Creating GloVe word embedding dictionary...')

    embeddings_index = {}
    f = open(Embedding_File, encoding='utf-8')

    for line in f:
        values = line.split()
        # word = values[0]
        word = ''.join(values[:-300])   
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found {} word vectors of glove.'.format(len(embeddings_index)))
    
if embedding_model == "Word2Vec":
    Embedding_Dim = 300 # Dimension of Word2Vec-embedding
    Embedding_File = 'GoogleNews-vectors-negative300.bin' # You may need to add the complete directory if the bin file is NOT in the same folder as this ipynb file
    print('Creating Word2Vec word embedding dictionary...')

    word2vec = KeyedVectors.load_word2vec_format(datapath(Embedding_File), binary=True)
    print('Found %s word vectors of word2vec' % len(word2vec))

if embedding_model == "Bert":
    Embedding_Dim = 1024 # Dimension of Bert-embedding
    if fine_tuned_model:
        bert_embeddings = BertEmbeddings(model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad')
    else:
        bert_embeddings = BertEmbeddings(model_name = 'bert-large-uncased-whole-word-masking')
    # This will create a tensor too large for a single computer, a more powerful one is needed
    '''
    embeddings_index = {}
    for word in word_index:
        output = bert_embeddings([word])
        for value in output[0]['embeddings_map'].values():
            embeddings_index[word] = np.array(value)
    print('Found {} word vectors of bert.'.format(len(embeddings_index)))
    '''
    # Use this instead
    embeddings_index = {}
    for word in word_index:
        output = bert_embeddings([word])
        result = np.array(output[0]['hidden_states'])[0][-1] + np.array(output[0]['hidden_states'])[0][-2] + np.array(output[0]['hidden_states'])[0][-3] + np.array(output[0]['hidden_states'])[0][-4]
        embeddings_index[word] = result
    print('Found {} word vectors of bert.'.format(len(embeddings_index)))
    
if embedding_model == "RoBerta":
    Embedding_Dim = 1024 # Dimension of Bert-embedding
    print('Creating RoBerta word embedding dictionary...')
    roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
    roberta_model = TFRobertaModel.from_pretrained('roberta-large')
    embeddings_index = {}
    for word in word_index:
        inputs = roberta_tokenizer(word, return_tensors="tf")
        outputs = roberta_model(inputs)
        result = np.array(outputs.last_hidden_state[0][1:-1])
        embeddings_index[word] = result
    print('Found {} word vectors of RoBerta.'.format(len(embeddings_index)))
    
if embedding_model == "DeBerta":
    Embedding_Dim = 1536 # Dimension of Bert-embedding
    print('Creating DeBerta word embedding dictionary...')
    deberta_tokenizer = DebertaV2Tokenizer.from_pretrained('kamalkraj/deberta-v2-xlarge')
    deberta_model = TFDebertaV2Model.from_pretrained('kamalkraj/deberta-v2-xlarge')
    embeddings_index = {}
    for word in word_index:
        inputs = deberta_tokenizer(word, return_tensors="tf")
        outputs = deberta_model(inputs)
        result = np.array(outputs.last_hidden_state[0][1:-1])
        embeddings_index[word] = result
    print('Found {} word vectors of DeBerta.'.format(len(embeddings_index)))

Creating GloVe word embedding dictionary...
Found 2195892 word vectors of glove.


In [285]:
leaks = df_train[['q1_q2_intersect', 'q1_freq', 'q2_freq']]
test_leaks = df_test[['q1_q2_intersect', 'q1_freq', 'q2_freq']]

ss = StandardScaler()
ss.fit(np.vstack((leaks, test_leaks)))
leaks = ss.transform(leaks)
test_leaks = ss.transform(test_leaks)

C:\Users\LTX_\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\LTX_\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [286]:
print(len(leaks))

311149


In [287]:
# optimized_feature_array = [1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]

In [288]:
# Add extra features input (optimal feature returned by XGBoost)
if use_more_features:
    #extra_features = df_train[['word_match_share', 'tfidf_word_match_share', 'tfidf_word_match', 'unigrams_common_count', 'unigrams_common_ratio',
    #                      'jaccard', 'common_words', 'common_words_stop', 'total_unique_words', 'total_unq_words_stop', 'wc_diff', 'wc_ratio', 
    #                      'wc_diff_unique', 'wc_ratio_unique', 'wc_diff_unique_stop', 'wc_ratio_unique_stop', 'same_start_word', 'char_diff', 
    #                      'char_ratio', 'char_diff_unique_stop']]
    extra_features = df_train[['word_match_share', 'tfidf_word_match', 'jaccard', 'total_unique_words', 'total_unq_words_stop', 'wc_diff', 'wc_ratio', 
                           'wc_diff_unique', 'wc_ratio_unique', 'same_start_word', 'char_diff', 'char_diff_unique_stop', 'q1_to_q2_wc_ratio_unique',
                              'q1_to_q2_char_diff', 'q1_to_q2_char_diff_unique_stop', 'word_match_share_alternative_stop', 'common_words_alternative_stop',
                              'total_unq_words_alternative_stop', 'wc_diff_unique_alternative_stop', 'char_diff_unique_alternative_stop',
                              'q1_to_q2_wc_diff_unique_alternative_stop', 'q1_to_q2_wc_ratio_unique_alternative_stop', 'q1_to_q2_char_diff_unique_alternative_stop']]
    #extra_features_test = df_test[['word_match_share', 'tfidf_word_match_share', 'tfidf_word_match', 'unigrams_common_count', 'unigrams_common_ratio',
    #                      'jaccard', 'common_words', 'common_words_stop', 'total_unique_words', 'total_unq_words_stop', 'wc_diff', 'wc_ratio', 
    #                      'wc_diff_unique', 'wc_ratio_unique', 'wc_diff_unique_stop', 'wc_ratio_unique_stop', 'same_start_word', 'char_diff', 
    #                      'char_ratio', 'char_diff_unique_stop']]
    extra_features_test = df_test[['word_match_share', 'tfidf_word_match', 'jaccard', 'total_unique_words', 'total_unq_words_stop', 'wc_diff', 'wc_ratio', 
                           'wc_diff_unique', 'wc_ratio_unique', 'same_start_word', 'char_diff', 'char_diff_unique_stop', 'q1_to_q2_wc_ratio_unique',
                              'q1_to_q2_char_diff', 'q1_to_q2_char_diff_unique_stop', 'word_match_share_alternative_stop', 'common_words_alternative_stop',
                              'total_unq_words_alternative_stop', 'wc_diff_unique_alternative_stop', 'char_diff_unique_alternative_stop',
                              'q1_to_q2_wc_diff_unique_alternative_stop', 'q1_to_q2_wc_ratio_unique_alternative_stop', 'q1_to_q2_char_diff_unique_alternative_stop']]
    
    ss = StandardScaler()
    ss.fit(np.vstack((extra_features, extra_features_test)))
    extra_features = ss.transform(extra_features)
    extra_features_test = ss.transform(extra_features_test)

In [289]:
# Create embedding matrix for embedding layer
print('Preparing embedding matrix')

num_words = min(Max_Num_Words, len(word_index))+1

embedding_matrix = np.zeros((num_words, Embedding_Dim))
if embedding_model == "GloVe" or embedding_model == "Bert":
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
if embedding_model == "Word2Vec":
    for word, i in word_index.items():
        try:
            embedding_vector = word2vec.get_vector(word)
            embedding_matrix[i] = embedding_vector
        except:
            continue
if embedding_model == "RoBerta" or embedding_model == "DeBerta":
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)[0]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
print('Null word embeddings: '.format(np.sum(np.sum(embedding_matrix, axis=1) == 0)))

Preparing embedding matrix
Null word embeddings: 


In [290]:
# Train Validation split
perm = np.random.permutation(len(train_data_1))
idx_train = perm[:int(len(train_data_1)*(1-Validation_Split_Ratio))]
idx_val = perm[int(len(train_data_1)*(1-Validation_Split_Ratio)):]

data_1_train = np.vstack((train_data_1[idx_train], train_data_2[idx_train]))
data_2_train = np.vstack((train_data_2[idx_train], train_data_1[idx_train]))
leaks_train = np.vstack((leaks[idx_train], leaks[idx_train]))
if use_more_features:
    feature_train = np.vstack((extra_features[idx_train], extra_features[idx_train]))
labels_train = np.concatenate((train_labels[idx_train], train_labels[idx_train]))

data_1_val = np.vstack((train_data_1[idx_val], train_data_2[idx_val]))
data_2_val = np.vstack((train_data_2[idx_val], train_data_1[idx_val]))
leaks_val = np.vstack((leaks[idx_val], leaks[idx_val]))
if use_more_features:
    feature_val = np.vstack((extra_features[idx_val], extra_features[idx_val]))
labels_val = np.concatenate((train_labels[idx_val], train_labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.471544715
    weight_val[labels_val==0] = 1.309033281

In [293]:
print(labels_train)

[0 1 1 ... 0 1 0]


In [11]:
pip uninstall tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

AttributeError: module 'tensorflow' has no attribute 'Session'

In [295]:
if neural_network == "LSTM":
    # The embedding layer containing the word vectors
    emb_layer = Embedding(
        input_dim=num_words,
        output_dim=Embedding_Dim,
        weights=[embedding_matrix],
        input_length=Max_Sequence_Length,
        trainable=False
    )    


    # LSTM layer

    lstm_layer = LSTM(Num_Lstm, dropout=Rate_Drop_Lstm, recurrent_dropout=Rate_Drop_Lstm)

    # Define inputs
    seq1 = Input(shape=(Max_Sequence_Length,), dtype='int32')
    seq2 = Input(shape=(Max_Sequence_Length,), dtype='int32')

    # Run inputs through embedding
    emb1 = emb_layer(seq1)
    emb2 = emb_layer(seq2)

    # Run through LSTM layers
    lstm_a = lstm_layer(emb1)
    # glob1a = GlobalAveragePooling1D()(lstm_a)
    lstm_b = lstm_layer(emb2)
    # glob1b = GlobalAveragePooling1D()(lstm_b)

    magic_input = Input(shape=(leaks.shape[1],))
    magic_dense = BatchNormalization()(magic_input)
    magic_dense = Dense(int(Num_Dense/2), activation=act_f)(magic_input)

    if use_more_features:
        feature_input = Input(shape=(extra_features.shape[1],))
        feature_dense = BatchNormalization()(feature_input)
        feature_dense = Dense(128, activation='relu')(feature_dense)

    if use_more_features:
        merged = concatenate([lstm_a, lstm_b, magic_dense, feature_dense])
    else:
        merged = concatenate([lstm_a, lstm_b, magic_dense])
    merged = BatchNormalization()(merged)
    merged = Dropout(Rate_Drop_Dense)(merged)

    merged = Dense(Num_Dense, activation=act_f)(merged)
    merged = BatchNormalization()(merged)
    merged = Dropout(Rate_Drop_Dense)(merged)

    preds = Dense(1, activation='sigmoid')(merged)

    # Add class weight
    if re_weight:
        class_weight = {0: 1.309033281, 1: 0.471544715}
    else:
        class_weight = None
        
    # Train the model
    if use_more_features:
        model = Model(inputs=[seq1, seq2, magic_input, feature_input], \
            outputs=preds)
    else:
        model = Model(inputs=[seq1, seq2, magic_input], \
            outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

    # Set early stopping (large patience should be useful)
    early_stopping =EarlyStopping(monitor='val_acc', patience=6)
    output_model_name = embedding_model + " + " + neural_network
    if use_more_features:
        output_model_name = output_model_name + "(extra features)"
    if embedding_model == "Bert" and fine_tuned_model:
        output_model_name = output_model_name + "(Bert fine_tunned)"
    output_model_name = output_model_name + '.h5'
    model_checkpoint = ModelCheckpoint(output_model_name, save_best_only=True, save_weights_only=True)

    if use_more_features:
        hist = model.fit([data_1_train, data_2_train, leaks_train, feature_train], labels_train, \
            validation_data=([data_1_val, data_2_val, leaks_val, feature_val], labels_val, weight_val), \
            epochs=200, batch_size=2048, shuffle=True, \
            class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])
    else:
        hist = model.fit([data_1_train, data_2_train, leaks_train], labels_train, \
            validation_data=([data_1_val, data_2_val, leaks_val], labels_val, weight_val), \
            epochs=200, batch_size=2048, shuffle=True, \
            class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])
    
if neural_network == "CNN":

    # The embedding layer containing the word vectors
    emb_layer = Embedding(
        input_dim=num_words,
        output_dim=Embedding_Dim,
        weights=[embedding_matrix],
        input_length=Max_Sequence_Length,
        trainable=False
    )

    # 1D convolutions that can iterate over the word vectors
    conv1 = Conv1D(filters=128, kernel_size=1, padding='same', activation='relu')
    conv2 = Conv1D(filters=128, kernel_size=2, padding='same', activation='relu')
    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')
    conv4 = Conv1D(filters=128, kernel_size=4, padding='same', activation='relu')
    conv5 = Conv1D(filters=32, kernel_size=5, padding='same', activation='relu')
    conv6 = Conv1D(filters=32, kernel_size=6, padding='same', activation='relu')

    # Define inputs
    seq1 = Input(shape=(60,))
    seq2 = Input(shape=(60,))

    # Run inputs through embedding
    emb1 = emb_layer(seq1)
    emb2 = emb_layer(seq2)

    # Run through CONV + GAP layers
    conv1a = conv1(emb1)
    glob1a = GlobalAveragePooling1D()(conv1a)
    conv1b = conv1(emb2)
    glob1b = GlobalAveragePooling1D()(conv1b)

    conv2a = conv2(emb1)
    glob2a = GlobalAveragePooling1D()(conv2a)
    conv2b = conv2(emb2)
    glob2b = GlobalAveragePooling1D()(conv2b)

    conv3a = conv3(emb1)
    glob3a = GlobalAveragePooling1D()(conv3a)
    conv3b = conv3(emb2)
    glob3b = GlobalAveragePooling1D()(conv3b)

    conv4a = conv4(emb1)
    glob4a = GlobalAveragePooling1D()(conv4a)
    conv4b = conv4(emb2)
    glob4b = GlobalAveragePooling1D()(conv4b)

    conv5a = conv5(emb1)
    glob5a = GlobalAveragePooling1D()(conv5a)
    conv5b = conv5(emb2)
    glob5b = GlobalAveragePooling1D()(conv5b)

    conv6a = conv6(emb1)
    glob6a = GlobalAveragePooling1D()(conv6a)
    conv6b = conv6(emb2)
    glob6b = GlobalAveragePooling1D()(conv6b)

    mergea = concatenate([glob1a, glob2a, glob3a, glob4a, glob5a, glob6a])
    mergeb = concatenate([glob1b, glob2b, glob3b, glob4b, glob5b, glob6b])

    diff = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=(4 * 128 + 2*32,))([mergea, mergeb])
    mul = Lambda(lambda x: x[0] * x[1], output_shape=(4 * 128 + 2*32,))([mergea, mergeb])

    magic_input = Input(shape=(leaks.shape[1],))
    magic_dense = BatchNormalization()(magic_input)
    magic_dense = Dense(64, activation='relu')(magic_dense)

    if use_more_features:
        feature_input = Input(shape=(extra_features.shape[1],))
        feature_dense = BatchNormalization()(feature_input)
        feature_dense = Dense(128, activation='relu')(feature_dense)

    if use_more_features:
        merge = concatenate([diff, mul, magic_dense, feature_dense])
    else:
        merge = concatenate([diff, mul, magic_dense])

    if re_weight:
        class_weight = {0: 1.309028344, 1: 0.472001959}
    else:
        class_weight = None

    # The MLP that determines the outcome
    x = Dropout(0.2)(merge)
    x = BatchNormalization()(x)
    x = Dense(300, activation='relu')(x)

    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation='sigmoid')(x)

    if use_more_features:
        model = Model(inputs=[seq1, seq2, magic_input, feature_input], outputs=pred)
    else:
        model = Model(inputs=[seq1, seq2, magic_input], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    # Set early stopping (large patience should be useful)
    early_stopping =EarlyStopping(monitor='val_acc', patience=6)
    output_model_name = embedding_model + " + " + neural_network
    if use_more_features:
        output_model_name = output_model_name + "(extra features)"
    if embedding_model == "Bert" and fine_tuned_model:
        output_model_name = output_model_name + "(Bert fine_tunned)"
    output_model_name = output_model_name + '.h5' 
    model_checkpoint = ModelCheckpoint(output_model_name, save_best_only=True, save_weights_only=True)

    if use_more_features:
        hist = model.fit([data_1_train, data_2_train, leaks_train, feature_train], labels_train, \
            validation_data=([data_1_val, data_2_val, leaks_val, feature_val], labels_val, weight_val), \
            epochs=200, batch_size=2048, shuffle=True, \
            class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])
    else:
        hist = model.fit([data_1_train, data_2_train, leaks_train], labels_train, \
            validation_data=([data_1_val, data_2_val, leaks_val], labels_val, weight_val), \
            epochs=200, batch_size=2048, shuffle=True, \
            class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

Epoch 1/200


C:\Users\LTX_\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


244/244 [==============================] - 2875s 12s/step - loss: 0.3999 - acc: 0.8270 - val_loss: 0.4530 - val_acc: 0.8163
Epoch 2/200
244/244 [==============================] - 2769s 11s/step - loss: 0.3411 - acc: 0.8518 - val_loss: 0.3753 - val_acc: 0.8265
Epoch 3/200
244/244 [==============================] - 2747s 11s/step - loss: 0.3237 - acc: 0.8578 - val_loss: 0.3201 - val_acc: 0.8578
Epoch 4/200
244/244 [==============================] - 2775s 11s/step - loss: 0.3113 - acc: 0.8625 - val_loss: 0.3114 - val_acc: 0.8611
Epoch 5/200
244/244 [==============================] - 2743s 11s/step - loss: 0.3007 - acc: 0.8669 - val_loss: 0.3166 - val_acc: 0.8563
Epoch 6/200
244/244 [==============================] - 3070s 13s/step - loss: 0.2917 - acc: 0.8708 - val_loss: 0.3067 - val_acc: 0.8617
Epoch 7/200
244/244 [==============================] - ETA: 0s - loss: 0.2826 - acc: 0.8748 WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not pr

KeyboardInterrupt: 

In [135]:
print(data_1_train)

[]


In [75]:
Model.compile(..., run_eagerly=True)

AttributeError: 'ellipsis' object has no attribute 'distribute_strategy'

In [ ]:
# Now the model training has finished. Save the model for future use
model.save(output_model_name) # store model parameters in .h5 file
bst_val_score = min(hist.history['val_acc'])